In [1]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display
from ipywidgets import interactive_output, HBox, VBox, Layout, FloatSlider, Label, HTMLMath

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace
from qiskit.visualization import array_to_latex, plot_bloch_vector
from qiskit.visualization.state_visualization import _bloch_multivector_data
from time import sleep

In [2]:
theta = Parameter('θ')
qc = QuantumCircuit(2)
qc.h(1)
qc.cry(theta,1,0);

In [3]:
def disp_cir(θ):
    qc_disp = qc.assign_parameters({theta:θ})
    display(qc_disp.draw(reverse_bits=True))

In [4]:
def plot_bloch(θ):

    qc_f = qc.assign_parameters({theta:θ})
    sv = Statevector(qc_f)
    bloch_data = _bloch_multivector_data(sv)

    width, height = plt.figaspect(1)
    figsize = (width, height)
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1, 1, 1, projection='3d')

    plot_bloch_vector(bloch_data, ax=ax, figsize=figsize)

In [5]:
def state_vector(θ):
    qc_s = qc.assign_parameters({theta:θ})
    sv_s = Statevector(qc_s)
    
    sv_lac = sv_s.draw('latex_source',prefix='|\\psi \\rangle = ',decimals=4)
    sv_html = HTMLMath(value = r'\(\color{black} {' + str(sv_lac)  + '}\)')
    display(sv_html)
    sleep(0.1) #Slows down the rendering so the display isn't so glitchy

In [6]:
def density_matrix(θ):
    qc_m = qc.assign_parameters({theta:θ})
    dm1 = partial_trace(DensityMatrix(qc_m),[0])
    dm_lac1 = array_to_latex(dm1,
                            precision=4,
                            source=True,
                            prefix='\\rho_A = ')

    dm2 = partial_trace(DensityMatrix(qc_m),[1])
    dm_lac2 = array_to_latex(dm2,
                            precision=4,
                            source=True,
                            prefix='\\rho_B = ')
    
    dm_html = HTMLMath(value = r'\(\color{#8296eb} {' + str(dm_lac1[0:len(dm_lac1)-0])  + '}\  \ \ \ \ \color{#c84d86} {' + str(dm_lac2[0:len(dm_lac2)-0])  + '}\)')
    display(dm_html)
    sleep(0.05) #Slows down the rendering so the display isn't so glitchy

In [7]:
θ_slide = FloatSlider(description="θ:",value=0.0,min=0.0,max=np.pi,step=np.pi/10)
plot_label = HTMLMath(value = r'\(\color{#8296eb} {\rho_A} \ \ \ \ \color{#c84d86} {\rho_B} \)')
cir_out = interactive_output(disp_cir,{'θ': θ_slide})
state_out = interactive_output(state_vector,{'θ': θ_slide})
den_out = interactive_output(density_matrix,{'θ': θ_slide})
bloch_out = interactive_output(plot_bloch,{'θ': θ_slide})
bloch_ball = HBox([VBox([plot_label,bloch_out],layout=Layout(align_items='center')),
                   VBox([θ_slide,cir_out,state_out,den_out],layout=Layout(width='500px'))])
bloch_ball